In [1111]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import re

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
lemmatizer = nltk.stem.WordNetLemmatizer()
wordnet_lemmatizer = WordNetLemmatizer()

%matplotlib inline

#  Merging dataframes of positive tweets with negative

In [1112]:
# importing initial dataset
df_1 = pd.read_csv('../data/raw/labeled_data.csv')

The following dataset had 6 columns:

1) count = number of CrowdFlower users who coded each tweet (min is 3, sometimes more users coded a tweet when judgments were determined to be unreliable by CF).

2) hate_speech = number of CF users who judged the tweet to be hate speech.

3) offensive_language = number of CF users who judged the tweet to be offensive.

4)neither = number of CF users who judged the tweet to be neither offensive nor non-offensive.

5)class = class label for majority of CF users. 0 - **hate speech** / 1 - **offensive language** / 2 - **neither**

6)tweet = content of the tweet

In [1113]:
# printing out the info
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24783 entries, 0 to 24782
Data columns (total 7 columns):
Unnamed: 0            24783 non-null int64
count                 24783 non-null int64
hate_speech           24783 non-null int64
offensive_language    24783 non-null int64
neither               24783 non-null int64
class                 24783 non-null int64
tweet                 24783 non-null object
dtypes: int64(6), object(1)
memory usage: 1.3+ MB


In [1114]:
# renaming the first columns as id
df_1.rename(columns={'Unnamed: 0':'id'}, inplace = True)
# increasing max length for all columns and number of columns
pd.set_option('display.max_colwidth', -1)
pd.set_option("display.max_columns", 50)

In [1115]:
# set id to index
df_1.set_index('id')

,count,hate_speech,offensive_language,neither,class,tweet
id,,,,,,
0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out...
1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn bad for cuffin dat hoe in the 1st place!!
2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby4life: You ever fuck a bitch and she start to cry? You be confused as shit
3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she look like a tranny
4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you hear about me might be true or it might be faker than the bitch who told it to ya &#57361;
...,...,...,...,...,...,...
25291,3,0,2,1,1,"you's a muthaf***in lie &#8220;@LifeAsKing: @20_Pearls @corey_emanuel right! His TL is trash &#8230;. Now, mine? Bible scriptures and hymns&#8221;"
25292,3,0,1,2,2,"you've gone and broke the wrong heart baby, and drove me redneck crazy"
25294,3,0,3,0,1,young buck wanna eat!!.. dat nigguh like I aint fuckin dis up again


In [1116]:
# there are three different classes on this dataset, all of which vaguely or clearly offensive
df_1['class'].value_counts()

1    19190
2    4163 
0    1430 
Name: class, dtype: int64

In the next section I'll take a closer look at some of the comments for each class in order to see whether the count/vote system is represantitive of the actual Tweet content

In [1117]:
# racist or homophobic comments
hate_speech = df_1.loc[df_1['class']==0] 

In [1118]:
hate_speech.head(50)


,id,count,hate_speech,offensive_language,neither,class,tweet
85,85,3,2,1,0,0,"""@Blackman38Tide: @WhaleLookyHere @HowdyDowdy11 queer"" gaywad"
89,90,3,3,0,0,0,"""@CB_Baby24: @white_thunduh alsarabsss"" hes a beaner smh you can tell hes a mexican"
110,111,3,3,0,0,0,"""@DevilGrimz: @VigxRArts you're fucking gay, blacklisted hoe"" Holding out for #TehGodClan anyway http://t.co/xUCcwoetmn"
184,186,3,3,0,0,0,"""@MarkRoundtreeJr: LMFAOOOO I HATE BLACK PEOPLE https://t.co/RNvD2nLCDR"" This is why there's black people and niggers"
202,204,3,2,1,0,0,"""@NoChillPaz: ""At least I'm not a nigger"" http://t.co/RGJa7CfoiT""\n\nLmfao"
204,206,3,2,1,0,0,"""@NotoriousBM95: @_WhitePonyJr_ Ariza is a snake and a coward"" but at least he isn't a cripple like your hero Roach lmaoo"
219,221,6,3,2,1,0,"""@RTNBA: Drakes new shoes that will be released by Nike/Jordan.... Yes, there's glitter on the shoes http://t.co/QCtPLxHEXM"" ....dudes a fag"
260,263,3,2,1,0,0,"""@TheoMaxximus: #GerrysHalloweenParty http://t.co/3ycrSrnjHc"" Halloween was yesterday stupid nigger"
312,317,3,2,1,0,0,"""@ashlingwilde: @ItsNotAdam is bored supposed to be cute, you faggot?"" Sometimes"
315,320,3,2,1,0,0,"""@bigbootybishopp: @white_thunduh lassen cc , the coach wants me"" i feel it youll get more my nig no doubt"


In [1119]:
# neither offensive nor non-offensive speech
neutral = df_1.loc[df_1['class']== 2] 

In [1120]:
neutral.head(50)

,id,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out...
40,40,3,0,1,2,2,""" momma said no pussy cats inside my doghouse """
63,63,3,0,0,3,2,"""@Addicted2Guys: -SimplyAddictedToGuys http://t.co/1jL4hi8ZMF"" woof woof hot scally lad"
66,66,3,0,1,2,2,"""@AllAboutManFeet: http://t.co/3gzUpfuMev"" woof woof and hot soles"
67,67,3,0,1,2,2,"""@Allyhaaaaa: Lemmie eat a Oreo &amp; do these dishes."" One oreo? Lol"
70,70,3,0,0,3,2,"""@ArizonasFinest6: Why the eggplant emoji doe?""y he say she looked like scream lmao"
75,75,3,0,1,2,2,"""@BabyAnimalPics: baby monkey bathtime http://t.co/7KPWAdLF0R""\nAwwwwe! This is soooo ADORABLE!"
115,116,3,0,0,3,2,"""@DomWorldPeace: Baseball season for the win. #Yankees"" This is where the love started"
118,119,3,0,0,3,2,"""@DunderbaIl: I'm an early bird and I'm a night owl, so I'm wise and have worms."""
119,120,3,0,1,2,2,"""@EdgarPixar: Overdosing on heavy drugs doesn't sound bad tonight."" I do that pussy shit every day."


In [1121]:
neutral['count'].value_counts()

3    3919
6    184 
4    37  
9    19  
7    4   
Name: count, dtype: int64

In [1122]:
# offensive but not hate speech
offensive_language = df_1.loc[df_1['class']==1] 

In [1123]:
offensive_language.head(50)

,id,count,hate_speech,offensive_language,neither,class,tweet
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn bad for cuffin dat hoe in the 1st place!!
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby4life: You ever fuck a bitch and she start to cry? You be confused as shit
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she look like a tranny
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you hear about me might be true or it might be faker than the bitch who told it to ya &#57361;
5,5,3,1,2,0,1,"!!!!!!!!!!!!!!!!!!""@T_Madison_x: The shit just blows me..claim you so faithful and down for somebody but still fucking with hoes! &#128514;&#128514;&#128514;"""
6,6,3,0,3,0,1,"!!!!!!""@__BrighterDays: I can not just sit up and HATE on another bitch .. I got too much shit going on!"""
7,7,3,0,3,0,1,!!!!&#8220;@selfiequeenbri: cause I'm tired of you big bitches coming for us skinny girls!!&#8221;
8,8,3,0,3,0,1,""" &amp; you might not get ya bitch back &amp; thats that """
9,9,3,1,2,0,1,""" @rhythmixx_ :hobbies include: fighting Mariam""\n\nbitch"
10,10,3,0,3,0,1,""" Keeks is a bitch she curves everyone "" lol I walked into a conversation like this. Smh"


In [1124]:
# inserting a new column in our dataframe to account for positive tweets (which we will extract in teh following section)
df_1.insert(5, 'positive' ,0)


In [1125]:
df_1['positive'].value_counts()


0    24783
Name: positive, dtype: int64

In [1126]:
df_1.tail()

,id,count,hate_speech,offensive_language,neither,positive,class,tweet
24778,25291,3,0,2,1,0,1,"you's a muthaf***in lie &#8220;@LifeAsKing: @20_Pearls @corey_emanuel right! His TL is trash &#8230;. Now, mine? Bible scriptures and hymns&#8221;"
24779,25292,3,0,1,2,0,2,"you've gone and broke the wrong heart baby, and drove me redneck crazy"
24780,25294,3,0,3,0,0,1,young buck wanna eat!!.. dat nigguh like I aint fuckin dis up again
24781,25295,6,0,6,0,0,1,youu got wild bitches tellin you lies
24782,25296,3,0,0,3,0,2,"~~Ruffled | Ntac Eileen Dahlia - Beautiful color combination of pink, orange, yellow &amp; white. A Coll http://t.co/H0dYEBvnZB"


At this point, we notice that our datapoints only contain either offensive language (of some form) or at least not non-offensive language labelled as neither. In order to build a model that can also successfully identify completely not offensive speech, we'll need to introduce some example of not-offensive language.

The following dataset has only 3 columns:

id: unique ID for each tweet

label: offensive or not

tweet: content of the tweet

In [1127]:
# importing a second dataset with only two types of tweets (offensive -1 / not offensive-0)
df_2 = pd.read_csv('../data/raw/train_E6oV3lV.csv')

In [1128]:
# only three columns 
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31962 entries, 0 to 31961
Data columns (total 3 columns):
id       31962 non-null int64
label    31962 non-null int64
tweet    31962 non-null object
dtypes: int64(2), object(1)
memory usage: 749.2+ KB


In [1129]:
# the majority of our datapoints are not offensive
df_2['label'].value_counts()

0    29720
1    2242 
Name: label, dtype: int64

In [1130]:
# given that we have 23,353 offensive tweets on the other dataset, we'll select 23,000 thousand positive tweets from df_2
df_pos = df_2.loc[df_2['label'] == 0]

In [1131]:
df_pos.head(50)

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run
1,2,0,@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦
4,5,0,factsguide: society now #motivation
5,6,0,[2/2] huge fan fare and big talking before they leave. chaos and pay disputes when they get there. #allshowandnogo
6,7,0,@user camping tomorrow @user @user @user @user @user @user @user dannyâ¦
7,8,0,the next school year is the year for exams.ð¯ can't think about that ð­ #school #exams #hate #imagine #actorslife #revolutionschool #girl
8,9,0,we won!!! love the land!!! #allin #cavs #champions #cleveland #clevelandcavaliers â¦
9,10,0,@user @user welcome here ! i'm it's so #gr8 !


In [1132]:
# selecting only 23000 to have a balanced dataset
df_p_23 = df_pos[:23000]

In [1133]:
len(df_p_23)

23000

In [1134]:
# setting the index as id
df_p_23.set_index('id')


,label,tweet
id,,
1,0,@user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run
2,0,@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked
3,0,bihday your majesty
4,0,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦
5,0,factsguide: society now #motivation
...,...,...
24725,0,"yes, leeds!! #lesbiunion #girlsweekend"
24726,0,in other non tattoo related news my boy will be #crystalpalace mascot at the home game against liverpool next season ! ð´ðµ
24727,0,finally wo agaya :):)


In [1135]:
# recreating the same structure of the other df
df_p_23.insert(1, 'count' ,3)
df_p_23.insert(2,'hate_speech',0)
df_p_23.insert(3,'offensive_language',0)
df_p_23.insert(4,'neither',0)
df_p_23.insert(5,'positive',3)

In [1136]:
df_p_23

,id,count,hate_speech,offensive_language,neither,positive,label,tweet
0,1,3,0,0,0,3,0,@user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run
1,2,3,0,0,0,3,0,@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked
2,3,3,0,0,0,3,0,bihday your majesty
3,4,3,0,0,0,3,0,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦
4,5,3,0,0,0,3,0,factsguide: society now #motivation
...,...,...,...,...,...,...,...,...
24724,24725,3,0,0,0,3,0,"yes, leeds!! #lesbiunion #girlsweekend"
24725,24726,3,0,0,0,3,0,in other non tattoo related news my boy will be #crystalpalace mascot at the home game against liverpool next season ! ð´ðµ
24726,24727,3,0,0,0,3,0,finally wo agaya :):)
24727,24728,3,0,0,0,3,0,ðð ð #love #instagood #photooftheday top.tags #tbt #cute #me #beautiful #followme #followâ¦


In [1137]:
# removing the existing label
df_p_23.drop(['label'], axis = 1, inplace = True)

In [1138]:
# inserting label as 4 for positive tweets
df_p_23.insert(6,'class', 3)


In [1139]:
df_p_23.head(10)

,id,count,hate_speech,offensive_language,neither,positive,class,tweet
0,1,3,0,0,0,3,3,@user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run
1,2,3,0,0,0,3,3,@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked
2,3,3,0,0,0,3,3,bihday your majesty
3,4,3,0,0,0,3,3,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦
4,5,3,0,0,0,3,3,factsguide: society now #motivation
5,6,3,0,0,0,3,3,[2/2] huge fan fare and big talking before they leave. chaos and pay disputes when they get there. #allshowandnogo
6,7,3,0,0,0,3,3,@user camping tomorrow @user @user @user @user @user @user @user dannyâ¦
7,8,3,0,0,0,3,3,the next school year is the year for exams.ð¯ can't think about that ð­ #school #exams #hate #imagine #actorslife #revolutionschool #girl
8,9,3,0,0,0,3,3,we won!!! love the land!!! #allin #cavs #champions #cleveland #clevelandcavaliers â¦
9,10,3,0,0,0,3,3,@user @user welcome here ! i'm it's so #gr8 !


In [1140]:
# making sure all Ids are unique. Last Id from df_1 was 25926
df_p_23['id'] = df_p_23['id'] + 25926

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [1141]:
# final df comprised of 23000 positive tweets
df_p_23.head(10)

,id,count,hate_speech,offensive_language,neither,positive,class,tweet
0,25927,3,0,0,0,3,3,@user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run
1,25928,3,0,0,0,3,3,@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked
2,25929,3,0,0,0,3,3,bihday your majesty
3,25930,3,0,0,0,3,3,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦
4,25931,3,0,0,0,3,3,factsguide: society now #motivation
5,25932,3,0,0,0,3,3,[2/2] huge fan fare and big talking before they leave. chaos and pay disputes when they get there. #allshowandnogo
6,25933,3,0,0,0,3,3,@user camping tomorrow @user @user @user @user @user @user @user dannyâ¦
7,25934,3,0,0,0,3,3,the next school year is the year for exams.ð¯ can't think about that ð­ #school #exams #hate #imagine #actorslife #revolutionschool #girl
8,25935,3,0,0,0,3,3,we won!!! love the land!!! #allin #cavs #champions #cleveland #clevelandcavaliers â¦
9,25936,3,0,0,0,3,3,@user @user welcome here ! i'm it's so #gr8 !


We'll then merge the negative plus positives tweets to have our initial unstructured dataset including both negative and positives tweets

In [1142]:
df = pd.concat([df_1, df_p_23], join = 'inner')

In [1143]:
df.set_index('id', inplace = True)

In [1144]:
# the final dataset has a good balance between positive and negatives tweets
df['positive'].value_counts()

0    24783
3    23000
Name: positive, dtype: int64

In [1145]:
df['positive'].value_counts()

0    24783
3    23000
Name: positive, dtype: int64

In [1146]:
# resshuffling the rows
df = df.sample(frac=1, random_state=42 ).reset_index(drop=True)

In [1147]:
# final df to be cleaned
len(df)

47783

In [1148]:
df['positive'].value_counts()

0    24783
3    23000
Name: positive, dtype: int64

# Data Cleaning

As first step we need to make all tweets lower case so that our model doesn't consider the same words with capital and lower initials as different.

In [1149]:
df['tweet_low'] = df['tweet'].apply(lambda x: x if type(x)!=str else x.lower())
df

,count,hate_speech,offensive_language,neither,positive,class,tweet,tweet_low
0,3,1,2,0,0,1,draymond green a bitch ass nigga bruh,draymond green a bitch ass nigga bruh
1,3,0,0,0,3,3,i decided to let you go after all. #reality,i decided to let you go after all. #reality
2,3,0,0,0,3,3,"another shooting after grimmie's incident. 50 dead. orlando, what's good? mas lisud pa guro kuhag drivers license ddto kysa palit baril","another shooting after grimmie's incident. 50 dead. orlando, what's good? mas lisud pa guro kuhag drivers license ddto kysa palit baril"
3,3,0,0,0,3,3,baptize my great nephew max tomorrow.,baptize my great nephew max tomorrow.
4,6,1,5,0,0,1,"RT @ovgenocide: First off, fuck your bitch. And the click you claim.","rt @ovgenocide: first off, fuck your bitch. and the click you claim."
...,...,...,...,...,...,...,...,...
47778,3,2,1,0,0,0,"I'm going to sell my iPhone and both my Macs, I don't support faggots.","i'm going to sell my iphone and both my macs, i don't support faggots."
47779,3,0,0,0,3,3,hello darkness my old friend. #scotland #sunset #hills #walking #goodbye,hello darkness my old friend. #scotland #sunset #hills #walking #goodbye
47780,3,0,0,0,3,3,i can't believe that my antidepressants are working. i didn't know if they were going to work or not.,i can't believe that my antidepressants are working. i didn't know if they were going to work or not.
47781,3,1,2,0,0,1,"#porn,#android,#iphone,#ipad,#sex,#xxx, | #Anal | Hardcore british queer anal pounding http://t.co/lRuEixMy21","#porn,#android,#iphone,#ipad,#sex,#xxx, | #anal | hardcore british queer anal pounding http://t.co/lrueixmy21"


##  Removing re-tweets

Hypelinks in this case represent retweets on each tweet.

In [1150]:
retweet =  df[df['tweet_low'].str.contains(r'http://t(?!$)')]
retweet

,count,hate_speech,offensive_language,neither,positive,class,tweet,tweet_low
5,6,1,5,0,0,1,RT @____Quinn: When you thinking about killing a bitch http://t.co/bT7hf28U0d,rt @____quinn: when you thinking about killing a bitch http://t.co/bt7hf28u0d
7,3,0,0,3,0,2,"'Scandal' sex scene airs on ABC right after 'It's the Great Pumpkin, Charlie Brown,' making parents angry http://t.co/vltSpK8U4S","'scandal' sex scene airs on abc right after 'it's the great pumpkin, charlie brown,' making parents angry http://t.co/vltspk8u4s"
8,3,0,3,0,0,1,"RT @khloewithoutheC: @TypicalMinions: when ur with ur friend and a bitch u both hate walks past http://t.co/LPLAYEfI7q"" @felferrara us as n&#8230;","rt @khloewithouthec: @typicalminions: when ur with ur friend and a bitch u both hate walks past http://t.co/lplayefi7q"" @felferrara us as n&#8230;"
17,3,1,0,2,0,2,CAIR Publishes a Hit List for Jihadis http://t.co/hvVfXypmHm via @RealJTP If you are a red blooded #American you may be on the list..,cair publishes a hit list for jihadis http://t.co/hvvfxypmhm via @realjtp if you are a red blooded #american you may be on the list..
23,6,0,6,0,0,1,"RT @keeLivinLavish: These bitches wanna fuck me, cuz dat molly got dem high&#128517;&#128138; http://t.co/xTnO13TnUS","rt @keelivinlavish: these bitches wanna fuck me, cuz dat molly got dem high&#128517;&#128138; http://t.co/xtno13tnus"
...,...,...,...,...,...,...,...,...
47742,3,0,3,0,0,1,RT @BrosConfessions: Don't be a pussy &#128076; http://t.co/J3USrLfo9A,rt @brosconfessions: don't be a pussy &#128076; http://t.co/j3usrlfo9a
47747,3,0,3,0,0,1,I'm sorry but this makes me feel so free. Like a fucking majestic bird. #gotonelessproblemwithoutya http://t.co/EKFOVVcWht,i'm sorry but this makes me feel so free. like a fucking majestic bird. #gotonelessproblemwithoutya http://t.co/ekfovvcwht
47752,3,0,3,0,0,1,RT @i8UrBitch: you ex bitches need to realize . http://t.co/1upHNo3iIP,rt @i8urbitch: you ex bitches need to realize . http://t.co/1uphno3iip
47771,3,0,3,0,0,1,RT @Killah_Kae: Will your bitches do this for you ? http://t.co/gmjkOLmcyQ,rt @killah_kae: will your bitches do this for you ? http://t.co/gmjkolmcyq


Only negative comments have retweets so I can't use retweets as a variable for our model

In [1151]:
retweet['class'].value_counts()

1    1754
2    883 
0    112 
Name: class, dtype: int64

Emojis come in form of text in the structure of an hashtag followed by only numeric digits (e.g #128532)

In [1152]:
emoji =  df[df['tweet'].str.contains(r'#[0-9]')]
emoji['class'].value_counts()

1    4852
2    994 
3    432 
0    213 
Name: class, dtype: int64

Within the positive tweets ashtag followed by numbers actually have linguistic meanings or represent dates/year

In [1153]:
positve_with_emoji = emoji.loc[emoji['class'] == 3]
positve_with_emoji.head(10)

,count,hate_speech,offensive_language,neither,positive,class,tweet,tweet_low
182,3,0,0,0,3,3,my dears! we had #12june #independentday to my #asgard for you-all i love you-all,my dears! we had #12june #independentday to my #asgard for you-all i love you-all
281,3,0,0,0,3,3,have a #16years â¤ï¸â¤ï¸â¤ï¸â¤ï¸â¤ï¸â¤ï¸â¤ï¸â¤ï¸â¤ï¸â¤ï¸â¤ï¸,have a #16years â¤ï¸â¤ï¸â¤ï¸â¤ï¸â¤ï¸â¤ï¸â¤ï¸â¤ï¸â¤ï¸â¤ï¸â¤ï¸
435,3,0,0,0,3,3,. happy bihday madokaâ£ï¸ . #19th #love #girls #bf #mu #0618,. happy bihday madokaâ£ï¸ . #19th #love #girls #bf #mu #0618
567,3,0,0,0,3,3,@user #33hr#school trip#enjoy #kyoto#kobeðð,@user #33hr#school trip#enjoy #kyoto#kobeðð
584,3,0,0,0,3,3,2nd my bihdayâð #2nd #my #bihday #glad #fun #surprise #thanks #bff #zk #å¤©çå¯º #mariottâ¦,2nd my bihdayâð #2nd #my #bihday #glad #fun #surprise #thanks #bff #zk #å¤©çå¯º #mariottâ¦
632,3,0,0,0,3,3,3rd anniversary bts btsday #3years with bts weâ¡you bts â¥rap monster â¥hobi â¥min suga â¥chim chim â¥tae tae â¥kookie,3rd anniversary bts btsday #3years with bts weâ¡you bts â¥rap monster â¥hobi â¥min suga â¥chim chim â¥tae tae â¥kookie
652,3,0,0,0,3,3,new music from #svvvgehouse aist : #tvbuu #g59 #suicideboys #houston #texas #soundcloud #666 #depression,new music from #svvvgehouse aist : #tvbuu #g59 #suicideboys #houston #texas #soundcloud #666 #depression
676,3,0,0,0,3,3,@user for india it's 5 weeks to go! @user #stevenspielberg ! #15july,@user for india it's 5 weeks to go! @user #stevenspielberg ! #15july
735,3,0,0,0,3,3,you're the pulse that i've always needed. â¤ #ilovevyka #myalways #10 #mygirl #inlove â¦,you're the pulse that i've always needed. â¤ #ilovevyka #myalways #10 #mygirl #inlove â¦
802,3,0,0,0,3,3,happy 21st bihday to me ðð #bihday #21 #years #june #cake #chocolate #candles,happy 21st bihday to me ðð #bihday #21 #years #june #cake #chocolate #candles


Once again the dataset of positive tweets was already stripped of emojis and we cannot include them as a variable towards our final model.

In [1154]:
# defining recurring function we will use to delete patterns from tweets
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt  

In [1155]:
# pattern to delete URLS for RETWEETS
df['no_url'] = np.vectorize(remove_pattern)(df['tweet_low'], "https?://[A-Za-z0-9./]*")

In [1156]:
# checking whether URLS have been removed
df

,count,hate_speech,offensive_language,neither,positive,class,tweet,tweet_low,no_url
0,3,1,2,0,0,1,draymond green a bitch ass nigga bruh,draymond green a bitch ass nigga bruh,draymond green a bitch ass nigga bruh
1,3,0,0,0,3,3,i decided to let you go after all. #reality,i decided to let you go after all. #reality,i decided to let you go after all. #reality
2,3,0,0,0,3,3,"another shooting after grimmie's incident. 50 dead. orlando, what's good? mas lisud pa guro kuhag drivers license ddto kysa palit baril","another shooting after grimmie's incident. 50 dead. orlando, what's good? mas lisud pa guro kuhag drivers license ddto kysa palit baril","another shooting after grimmie's incident. 50 dead. orlando, what's good? mas lisud pa guro kuhag drivers license ddto kysa palit baril"
3,3,0,0,0,3,3,baptize my great nephew max tomorrow.,baptize my great nephew max tomorrow.,baptize my great nephew max tomorrow.
4,6,1,5,0,0,1,"RT @ovgenocide: First off, fuck your bitch. And the click you claim.","rt @ovgenocide: first off, fuck your bitch. and the click you claim.","rt @ovgenocide: first off, fuck your bitch. and the click you claim."
...,...,...,...,...,...,...,...,...,...
47778,3,2,1,0,0,0,"I'm going to sell my iPhone and both my Macs, I don't support faggots.","i'm going to sell my iphone and both my macs, i don't support faggots.","i'm going to sell my iphone and both my macs, i don't support faggots."
47779,3,0,0,0,3,3,hello darkness my old friend. #scotland #sunset #hills #walking #goodbye,hello darkness my old friend. #scotland #sunset #hills #walking #goodbye,hello darkness my old friend. #scotland #sunset #hills #walking #goodbye
47780,3,0,0,0,3,3,i can't believe that my antidepressants are working. i didn't know if they were going to work or not.,i can't believe that my antidepressants are working. i didn't know if they were going to work or not.,i can't believe that my antidepressants are working. i didn't know if they were going to work or not.
47781,3,1,2,0,0,1,"#porn,#android,#iphone,#ipad,#sex,#xxx, | #Anal | Hardcore british queer anal pounding http://t.co/lRuEixMy21","#porn,#android,#iphone,#ipad,#sex,#xxx, | #anal | hardcore british queer anal pounding http://t.co/lrueixmy21","#porn,#android,#iphone,#ipad,#sex,#xxx, | #anal | hardcore british queer anal pounding"


## Removing Twitter handles and count them

In [1157]:
def count_handles (input_txt, pattern):
    
    r = re.findall(pattern, input_txt)
    return len(r)
        

In [1158]:
df['handle_count'] = np.vectorize(count_handles)(df['tweet_low'], "@[\w]*")

In [1159]:
df.head(5)

,count,hate_speech,offensive_language,neither,positive,class,tweet,tweet_low,no_url,handle_count
0,3,1,2,0,0,1,draymond green a bitch ass nigga bruh,draymond green a bitch ass nigga bruh,draymond green a bitch ass nigga bruh,0
1,3,0,0,0,3,3,i decided to let you go after all. #reality,i decided to let you go after all. #reality,i decided to let you go after all. #reality,0
2,3,0,0,0,3,3,"another shooting after grimmie's incident. 50 dead. orlando, what's good? mas lisud pa guro kuhag drivers license ddto kysa palit baril","another shooting after grimmie's incident. 50 dead. orlando, what's good? mas lisud pa guro kuhag drivers license ddto kysa palit baril","another shooting after grimmie's incident. 50 dead. orlando, what's good? mas lisud pa guro kuhag drivers license ddto kysa palit baril",0
3,3,0,0,0,3,3,baptize my great nephew max tomorrow.,baptize my great nephew max tomorrow.,baptize my great nephew max tomorrow.,0
4,6,1,5,0,0,1,"RT @ovgenocide: First off, fuck your bitch. And the click you claim.","rt @ovgenocide: first off, fuck your bitch. and the click you claim.","rt @ovgenocide: first off, fuck your bitch. and the click you claim.",1


In [1160]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt    

In [1161]:
# remove twitter handles (@user)
df['no_handle'] = np.vectorize(remove_pattern)(df['no_url'], "@[\w]*")

In [1162]:
df.tail(10)

,count,hate_speech,offensive_language,neither,positive,class,tweet,tweet_low,no_url,handle_count,no_handle
47773,3,0,0,0,3,3,bad news for journalism when my monthly neighborhood newsletter is down to 4 pages and 2 is ads. !,bad news for journalism when my monthly neighborhood newsletter is down to 4 pages and 2 is ads. !,bad news for journalism when my monthly neighborhood newsletter is down to 4 pages and 2 is ads. !,0,bad news for journalism when my monthly neighborhood newsletter is down to 4 pages and 2 is ads. !
47774,6,0,6,0,0,1,This bitch has two jobs and is starting school on the 26th. #fuckwhatyouheard,this bitch has two jobs and is starting school on the 26th. #fuckwhatyouheard,this bitch has two jobs and is starting school on the 26th. #fuckwhatyouheard,0,this bitch has two jobs and is starting school on the 26th. #fuckwhatyouheard
47775,3,0,0,0,3,3,just listened to @user obama speech re orlando killings -he completely politicized the event instead of addressing the tragedy itself.,just listened to @user obama speech re orlando killings -he completely politicized the event instead of addressing the tragedy itself.,just listened to @user obama speech re orlando killings -he completely politicized the event instead of addressing the tragedy itself.,1,just listened to obama speech re orlando killings -he completely politicized the event instead of addressing the tragedy itself.
47776,3,0,3,0,0,1,RT @PRAYINGFORHEAD: tyga used to be good before he got into this swag fag shit,rt @prayingforhead: tyga used to be good before he got into this swag fag shit,rt @prayingforhead: tyga used to be good before he got into this swag fag shit,1,rt : tyga used to be good before he got into this swag fag shit
47777,3,0,3,0,0,1,@jkatz_ @kaylee_binion14 bitch. &#128534;,@jkatz_ @kaylee_binion14 bitch. &#128534;,@jkatz_ @kaylee_binion14 bitch. &#128534;,2,bitch. &#128534;
47778,3,2,1,0,0,0,"I'm going to sell my iPhone and both my Macs, I don't support faggots.","i'm going to sell my iphone and both my macs, i don't support faggots.","i'm going to sell my iphone and both my macs, i don't support faggots.",0,"i'm going to sell my iphone and both my macs, i don't support faggots."
47779,3,0,0,0,3,3,hello darkness my old friend. #scotland #sunset #hills #walking #goodbye,hello darkness my old friend. #scotland #sunset #hills #walking #goodbye,hello darkness my old friend. #scotland #sunset #hills #walking #goodbye,0,hello darkness my old friend. #scotland #sunset #hills #walking #goodbye
47780,3,0,0,0,3,3,i can't believe that my antidepressants are working. i didn't know if they were going to work or not.,i can't believe that my antidepressants are working. i didn't know if they were going to work or not.,i can't believe that my antidepressants are working. i didn't know if they were going to work or not.,0,i can't believe that my antidepressants are working. i didn't know if they were going to work or not.
47781,3,1,2,0,0,1,"#porn,#android,#iphone,#ipad,#sex,#xxx, | #Anal | Hardcore british queer anal pounding http://t.co/lRuEixMy21","#porn,#android,#iphone,#ipad,#sex,#xxx, | #anal | hardcore british queer anal pounding http://t.co/lrueixmy21","#porn,#android,#iphone,#ipad,#sex,#xxx, | #anal | hardcore british queer anal pounding",0,"#porn,#android,#iphone,#ipad,#sex,#xxx, | #anal | hardcore british queer anal pounding"
47782,3,0,3,0,0,1,"RT @JennyJohnsonHi5: Just when I thought Justin Bieber couldn't be anymore of a pussy, he gets arrested in Canada for fighting a person who&#8230;","rt @jennyjohnsonhi5: just when i thought justin bieber couldn't be anymore of a pussy, he gets arrested in canada for fighting a person who&#8230;","rt @jennyjohnsonhi5: just when i thought justin bieber couldn't be anymore of a pussy, he gets arrested in canada for fighting a person who&#8230;",1,"rt : just when i thought justin bieber couldn't be anymore of a pussy, he gets arrested in canada for fighting a person who&#8230;"


## Removing special characters apart from hashtags and counting tweet lenght

Also, deleting all special characters apart from hastags followed by words since emojis are only applicable to bad tweets.

In [1163]:
# getting rid of all characters apart from letters ashtags and apostrophes
df['no_handle_no_special'] = df['no_handle'].str.replace("[^a-zA-Z#']", " ")

In [1164]:
df

,count,hate_speech,offensive_language,neither,positive,class,tweet,tweet_low,no_url,handle_count,no_handle,no_handle_no_special
0,3,1,2,0,0,1,draymond green a bitch ass nigga bruh,draymond green a bitch ass nigga bruh,draymond green a bitch ass nigga bruh,0,draymond green a bitch ass nigga bruh,draymond green a bitch ass nigga bruh
1,3,0,0,0,3,3,i decided to let you go after all. #reality,i decided to let you go after all. #reality,i decided to let you go after all. #reality,0,i decided to let you go after all. #reality,i decided to let you go after all #reality
2,3,0,0,0,3,3,"another shooting after grimmie's incident. 50 dead. orlando, what's good? mas lisud pa guro kuhag drivers license ddto kysa palit baril","another shooting after grimmie's incident. 50 dead. orlando, what's good? mas lisud pa guro kuhag drivers license ddto kysa palit baril","another shooting after grimmie's incident. 50 dead. orlando, what's good? mas lisud pa guro kuhag drivers license ddto kysa palit baril",0,"another shooting after grimmie's incident. 50 dead. orlando, what's good? mas lisud pa guro kuhag drivers license ddto kysa palit baril",another shooting after grimmie's incident dead orlando what's good mas lisud pa guro kuhag drivers license ddto kysa palit baril
3,3,0,0,0,3,3,baptize my great nephew max tomorrow.,baptize my great nephew max tomorrow.,baptize my great nephew max tomorrow.,0,baptize my great nephew max tomorrow.,baptize my great nephew max tomorrow
4,6,1,5,0,0,1,"RT @ovgenocide: First off, fuck your bitch. And the click you claim.","rt @ovgenocide: first off, fuck your bitch. and the click you claim.","rt @ovgenocide: first off, fuck your bitch. and the click you claim.",1,"rt : first off, fuck your bitch. and the click you claim.",rt first off fuck your bitch and the click you claim
...,...,...,...,...,...,...,...,...,...,...,...,...
47778,3,2,1,0,0,0,"I'm going to sell my iPhone and both my Macs, I don't support faggots.","i'm going to sell my iphone and both my macs, i don't support faggots.","i'm going to sell my iphone and both my macs, i don't support faggots.",0,"i'm going to sell my iphone and both my macs, i don't support faggots.",i'm going to sell my iphone and both my macs i don't support faggots
47779,3,0,0,0,3,3,hello darkness my old friend. #scotland #sunset #hills #walking #goodbye,hello darkness my old friend. #scotland #sunset #hills #walking #goodbye,hello darkness my old friend. #scotland #sunset #hills #walking #goodbye,0,hello darkness my old friend. #scotland #sunset #hills #walking #goodbye,hello darkness my old friend #scotland #sunset #hills #walking #goodbye
47780,3,0,0,0,3,3,i can't believe that my antidepressants are working. i didn't know if they were going to work or not.,i can't believe that my antidepressants are working. i didn't know if they were going to work or not.,i can't believe that my antidepressants are working. i didn't know if they were going to work or not.,0,i can't believe that my antidepressants are working. i didn't know if they were going to work or not.,i can't believe that my antidepressants are working i didn't know if they were going to work or not
47781,3,1,2,0,0,1,"#porn,#android,#iphone,#ipad,#sex,#xxx, | #Anal | Hardcore british queer anal pounding http://t.co/lRuEixMy21","#porn,#android,#iphone,#ipad,#sex,#xxx, | #anal | hardcore british queer anal pounding http://t.co/lrueixmy21","#porn,#android,#iphone,#ipad,#sex,#xxx, | #anal | hardcore british queer anal pounding",0,"#porn,#android,#iphone,#ipad,#sex,#xxx, | #anal | hardcore british queer anal pounding",#porn #android #iphone #ipad #sex #xxx #anal hardcore british queer anal pounding


In [1165]:
# remove single ashtags with nothing following them
df['no_handle_no_special_no_sin_ash'] = np.vectorize(remove_pattern)(df['no_handle_no_special'], " # ")

In [1166]:
# counting the length of tweets after URLs are removed, we will use this to see if there's a correlation between length of tweets and sentiment
df['tweet_length'] = df['no_handle_no_special_no_sin_ash'].apply(lambda x: len(x))
df.head(50)

,count,hate_speech,offensive_language,neither,positive,class,tweet,tweet_low,no_url,handle_count,no_handle,no_handle_no_special,no_handle_no_special_no_sin_ash,tweet_length
0,3,1,2,0,0,1,draymond green a bitch ass nigga bruh,draymond green a bitch ass nigga bruh,draymond green a bitch ass nigga bruh,0,draymond green a bitch ass nigga bruh,draymond green a bitch ass nigga bruh,draymond green a bitch ass nigga bruh,37
1,3,0,0,0,3,3,i decided to let you go after all. #reality,i decided to let you go after all. #reality,i decided to let you go after all. #reality,0,i decided to let you go after all. #reality,i decided to let you go after all #reality,i decided to let you go after all #reality,45
2,3,0,0,0,3,3,"another shooting after grimmie's incident. 50 dead. orlando, what's good? mas lisud pa guro kuhag drivers license ddto kysa palit baril","another shooting after grimmie's incident. 50 dead. orlando, what's good? mas lisud pa guro kuhag drivers license ddto kysa palit baril","another shooting after grimmie's incident. 50 dead. orlando, what's good? mas lisud pa guro kuhag drivers license ddto kysa palit baril",0,"another shooting after grimmie's incident. 50 dead. orlando, what's good? mas lisud pa guro kuhag drivers license ddto kysa palit baril",another shooting after grimmie's incident dead orlando what's good mas lisud pa guro kuhag drivers license ddto kysa palit baril,another shooting after grimmie's incident dead orlando what's good mas lisud pa guro kuhag drivers license ddto kysa palit baril,137
3,3,0,0,0,3,3,baptize my great nephew max tomorrow.,baptize my great nephew max tomorrow.,baptize my great nephew max tomorrow.,0,baptize my great nephew max tomorrow.,baptize my great nephew max tomorrow,baptize my great nephew max tomorrow,39
4,6,1,5,0,0,1,"RT @ovgenocide: First off, fuck your bitch. And the click you claim.","rt @ovgenocide: first off, fuck your bitch. and the click you claim.","rt @ovgenocide: first off, fuck your bitch. and the click you claim.",1,"rt : first off, fuck your bitch. and the click you claim.",rt first off fuck your bitch and the click you claim,rt first off fuck your bitch and the click you claim,57
5,6,1,5,0,0,1,RT @____Quinn: When you thinking about killing a bitch http://t.co/bT7hf28U0d,rt @____quinn: when you thinking about killing a bitch http://t.co/bt7hf28u0d,rt @____quinn: when you thinking about killing a bitch,1,rt : when you thinking about killing a bitch,rt when you thinking about killing a bitch,rt when you thinking about killing a bitch,45
6,3,0,0,0,3,3,#psychotherapy blog: redefining what makes you,#psychotherapy blog: redefining what makes you,#psychotherapy blog: redefining what makes you,0,#psychotherapy blog: redefining what makes you,#psychotherapy blog redefining what makes you,#psychotherapy blog redefining what makes you,49
7,3,0,0,3,0,2,"'Scandal' sex scene airs on ABC right after 'It's the Great Pumpkin, Charlie Brown,' making parents angry http://t.co/vltSpK8U4S","'scandal' sex scene airs on abc right after 'it's the great pumpkin, charlie brown,' making parents angry http://t.co/vltspk8u4s","'scandal' sex scene airs on abc right after 'it's the great pumpkin, charlie brown,' making parents angry",0,"'scandal' sex scene airs on abc right after 'it's the great pumpkin, charlie brown,' making parents angry",'scandal' sex scene airs on abc right after 'it's the great pumpkin charlie brown ' making parents angry,'scandal' sex scene airs on abc right after 'it's the great pumpkin charlie brown ' making parents angry,106
8,3,0,3,0,0,1,"RT @khloewithoutheC: @TypicalMinions: when ur with ur friend and a bitch u both hate walks past http://t.co/LPLAYEfI7q"" @felferrara us as n&#8230;","rt @khloewithouthec: @typicalminions: when ur with ur friend and a bitch u both hate walks past http://t.co/lplayefi7q"" @felferrara us as n&#8230;","rt @khloewithouthec: @typicalminions: when ur with ur friend and a bitch u both hate walks past "" @felferrara us as n&#8230;",3,"rt : : when u

Checking there are no outliers mistake as max length for Tweets should be **280**

In [1167]:
df_sub = df.loc[df['tweet_length']>280]
# df_sub =df['no_handle'].str.replace("[^a-zA-Z#]", " ")

The longest twitter has 511 according to our previous lambda function. However, this seems to be off as the actual tweet is significantly shorter than that.

In [1168]:
df_sub.sort_values(by=['tweet_length'], ascending = False)

,count,hate_speech,offensive_language,neither,positive,class,tweet,tweet_low,no_url,handle_count,no_handle,no_handle_no_special,no_handle_no_special_no_sin_ash,tweet_length
23178,3,0,3,0,0,1,RT @TrxllLegend: One good girl is worth a thousand bitches\n\n&#128112; = &#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#8230;,rt @trxlllegend: one good girl is worth a thousand bitches\n\n&#128112; = &#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#8230;,rt @trxlllegend: one good girl is worth a thousand bitches\n\n&#128112; = &#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#8230;,1,rt : one good girl is worth a thousand bitches\n\n&#128112; = &#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#8230;,rt one good girl is worth a thousand bitches # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #,rt one good girl is worth a thousand bitches,511
18367,3,0,3,0,0,1,No summer school? &#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#1

There seems to be something off with the spacing in each tweet. Removing all spaces within characters.

In [1169]:
# removing spaces in tweets to count only characters
df['nospaces'] = df['no_handle_no_special_no_sin_ash'].str.replace(" ", "")

In [1170]:
df

,count,hate_speech,offensive_language,neither,positive,class,tweet,tweet_low,no_url,handle_count,no_handle,no_handle_no_special,no_handle_no_special_no_sin_ash,tweet_length,nospaces
0,3,1,2,0,0,1,draymond green a bitch ass nigga bruh,draymond green a bitch ass nigga bruh,draymond green a bitch ass nigga bruh,0,draymond green a bitch ass nigga bruh,draymond green a bitch ass nigga bruh,draymond green a bitch ass nigga bruh,37,draymondgreenabitchassniggabruh
1,3,0,0,0,3,3,i decided to let you go after all. #reality,i decided to let you go after all. #reality,i decided to let you go after all. #reality,0,i decided to let you go after all. #reality,i decided to let you go after all #reality,i decided to let you go after all #reality,45,idecidedtoletyougoafterall#reality
2,3,0,0,0,3,3,"another shooting after grimmie's incident. 50 dead. orlando, what's good? mas lisud pa guro kuhag drivers license ddto kysa palit baril","another shooting after grimmie's incident. 50 dead. orlando, what's good? mas lisud pa guro kuhag drivers license ddto kysa palit baril","another shooting after grimmie's incident. 50 dead. orlando, what's good? mas lisud pa guro kuhag drivers license ddto kysa palit baril",0,"another shooting after grimmie's incident. 50 dead. orlando, what's good? mas lisud pa guro kuhag drivers license ddto kysa palit baril",another shooting after grimmie's incident dead orlando what's good mas lisud pa guro kuhag drivers license ddto kysa palit baril,another shooting after grimmie's incident dead orlando what's good mas lisud pa guro kuhag drivers license ddto kysa palit baril,137,anothershootingaftergrimmie'sincidentdeadorlandowhat'sgoodmaslisudpagurokuhagdriverslicenseddtokysapalitbaril
3,3,0,0,0,3,3,baptize my great nephew max tomorrow.,baptize my great nephew max tomorrow.,baptize my great nephew max tomorrow.,0,baptize my great nephew max tomorrow.,baptize my great nephew max tomorrow,baptize my great nephew max tomorrow,39,baptizemygreatnephewmaxtomorrow
4,6,1,5,0,0,1,"RT @ovgenocide: First off, fuck your bitch. And the click you claim.","rt @ovgenocide: first off, fuck your bitch. and the click you claim.","rt @ovgenocide: first off, fuck your bitch. and the click you claim.",1,"rt : first off, fuck your bitch. and the click you claim.",rt first off fuck your bitch and the click you claim,rt first off fuck your bitch and the click you claim,57,rtfirstofffuckyourbitchandtheclickyouclaim
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47778,3,2,1,0,0,0,"I'm going to sell my iPhone and both my Macs, I don't support faggots.","i'm going to sell my iphone and both my macs, i don't support faggots.","i'm going to sell my iphone and both my macs, i don't support faggots.",0,"i'm going to sell my iphone and both my macs, i don't support faggots.",i'm going to sell my iphone and both my macs i don't support faggots,i'm going to sell my iphone and both my macs i don't support faggots,70,i'mgoingtosellmyiphoneandbothmymacsidon'tsupportfaggots
47779,3,0,0,0,3,3,hello darkness my old friend. #scotland #sunset #hills #walking #goodbye,hello darkness my old friend. #scotland #sunset #hills #walking #goodbye,hello darkness my old friend. #scotland #sunset #hills #walking #goodbye,0,hello darkness my old friend. #scotland #sunset #hills #walking #goodbye,hello darkness my old friend #scotland #sunset #hills #walking #goodbye,hello darkness my old friend #scotland #sunset #hills #walking #goodbye,77,hellodarknessmyoldfriend#scotland#sunset#hills#walking#goodbye
47780,3,0,0,0,3,3,i can't believe that my antidepressants are working. i didn't know if they were going to work or not.,i can't believe that my antidepressants are working. i didn't know if they were going to work or not.,i can't believe that my antidepressants are working. i didn't know if they were going to work or not.,0,i can't believe that my antidepressants are working. i didn't know if they were going to work or not.,i can't believe that my antidepressants ar

In [1171]:
df['character_count'] = df['nospaces'].apply(lambda x: len(x))
df.head(50)

,count,hate_speech,offensive_language,neither,positive,class,tweet,tweet_low,no_url,handle_count,no_handle,no_handle_no_special,no_handle_no_special_no_sin_ash,tweet_length,nospaces,character_count
0,3,1,2,0,0,1,draymond green a bitch ass nigga bruh,draymond green a bitch ass nigga bruh,draymond green a bitch ass nigga bruh,0,draymond green a bitch ass nigga bruh,draymond green a bitch ass nigga bruh,draymond green a bitch ass nigga bruh,37,draymondgreenabitchassniggabruh,31
1,3,0,0,0,3,3,i decided to let you go after all. #reality,i decided to let you go after all. #reality,i decided to let you go after all. #reality,0,i decided to let you go after all. #reality,i decided to let you go after all #reality,i decided to let you go after all #reality,45,idecidedtoletyougoafterall#reality,34
2,3,0,0,0,3,3,"another shooting after grimmie's incident. 50 dead. orlando, what's good? mas lisud pa guro kuhag drivers license ddto kysa palit baril","another shooting after grimmie's incident. 50 dead. orlando, what's good? mas lisud pa guro kuhag drivers license ddto kysa palit baril","another shooting after grimmie's incident. 50 dead. orlando, what's good? mas lisud pa guro kuhag drivers license ddto kysa palit baril",0,"another shooting after grimmie's incident. 50 dead. orlando, what's good? mas lisud pa guro kuhag drivers license ddto kysa palit baril",another shooting after grimmie's incident dead orlando what's good mas lisud pa guro kuhag drivers license ddto kysa palit baril,another shooting after grimmie's incident dead orlando what's good mas lisud pa guro kuhag drivers license ddto kysa palit baril,137,anothershootingaftergrimmie'sincidentdeadorlandowhat'sgoodmaslisudpagurokuhagdriverslicenseddtokysapalitbaril,109
3,3,0,0,0,3,3,baptize my great nephew max tomorrow.,baptize my great nephew max tomorrow.,baptize my great nephew max tomorrow.,0,baptize my great nephew max tomorrow.,baptize my great nephew max tomorrow,baptize my great nephew max tomorrow,39,baptizemygreatnephewmaxtomorrow,31
4,6,1,5,0,0,1,"RT @ovgenocide: First off, fuck your bitch. And the click you claim.","rt @ovgenocide: first off, fuck your bitch. and the click you claim.","rt @ovgenocide: first off, fuck your bitch. and the click you claim.",1,"rt : first off, fuck your bitch. and the click you claim.",rt first off fuck your bitch and the click you claim,rt first off fuck your bitch and the click you claim,57,rtfirstofffuckyourbitchandtheclickyouclaim,42
5,6,1,5,0,0,1,RT @____Quinn: When you thinking about killing a bitch http://t.co/bT7hf28U0d,rt @____quinn: when you thinking about killing a bitch http://t.co/bt7hf28u0d,rt @____quinn: when you thinking about killing a bitch,1,rt : when you thinking about killing a bitch,rt when you thinking about killing a bitch,rt when you thinking about killing a bitch,45,rtwhenyouthinkingaboutkillingabitch,35
6,3,0,0,0,3,3,#psychotherapy blog: redefining what makes you,#psychotherapy blog: redefining what makes you,#psychotherapy blog: redefining what makes you,0,#psychotherapy blog: redefining what makes you,#psychotherapy blog redefining what makes you,#psychotherapy blog redefining what makes you,49,#psychotherapyblogredefiningwhatmakesyou,40
7,3,0,0,3,0,2,"'Scandal' sex scene airs on ABC right after 'It's the Great Pumpkin, Charlie Brown,' making parents angry http://t.co/vltSpK8U4S","'scandal' sex scene airs on abc right after 'it's the great pumpkin, charlie brown,' making parents angry http://t.co/vltspk8u4s","'scandal' sex scene airs on abc right after 'it's the great pumpkin, charlie brown,' making parents angry",0,"'scandal' sex scene airs on abc right after 'it's the great pumpkin, charlie brown,' making parents angry",'scandal' sex scene airs on abc right after 'it's the great pumpkin charlie brown ' making parents angry,'scandal' sex scene airs on abc right after 'it's the great pumpkin charlie brown ' making parents angry,106,'scandal'sexsceneairsonabcrightafter'it'sthegreatpumpkincharliebrown'makingparents

In [1172]:
# double checking issue is fixed with the longest tweet from previous length
df.iloc[23178]

count                              3                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
hate_speech                        0                                                                                                                                                                            

In [1173]:
df.isna().sum()

count                              0
hate_speech                        0
offensive_language                 0
neither                            0
positive                           0
class                              0
tweet                              0
tweet_low                          0
no_url                             0
handle_count                       0
no_handle                          0
no_handle_no_special               0
no_handle_no_special_no_sin_ash    0
tweet_length                       0
nospaces                           0
character_count                    0
dtype: int64

##  Lemmatization with POS

In [1174]:
# defining different POS
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

# function to lemmatize all words in a sentence
def lemmatize_sentence(sentence):
    #tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
            # 'ass' kept being reduced to 'as' for some reason         
        if word == 'ass':
            lemmatized_sentence.append(word)
        
        elif tag is None:
            #if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:
            #else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return " ".join(lemmatized_sentence)


In [1175]:
df['lemmatized'] = df['no_handle_no_special_no_sin_ash'].apply(lambda x: lemmatize_sentence(x))

In [1176]:
df.head(10)

,count,hate_speech,offensive_language,neither,positive,class,tweet,tweet_low,no_url,handle_count,no_handle,no_handle_no_special,no_handle_no_special_no_sin_ash,tweet_length,nospaces,character_count,lemmatized
0,3,1,2,0,0,1,draymond green a bitch ass nigga bruh,draymond green a bitch ass nigga bruh,draymond green a bitch ass nigga bruh,0,draymond green a bitch ass nigga bruh,draymond green a bitch ass nigga bruh,draymond green a bitch ass nigga bruh,37,draymondgreenabitchassniggabruh,31,draymond green a bitch ass nigga bruh
1,3,0,0,0,3,3,i decided to let you go after all. #reality,i decided to let you go after all. #reality,i decided to let you go after all. #reality,0,i decided to let you go after all. #reality,i decided to let you go after all #reality,i decided to let you go after all #reality,45,idecidedtoletyougoafterall#reality,34,i decide to let you go after all # reality
2,3,0,0,0,3,3,"another shooting after grimmie's incident. 50 dead. orlando, what's good? mas lisud pa guro kuhag drivers license ddto kysa palit baril","another shooting after grimmie's incident. 50 dead. orlando, what's good? mas lisud pa guro kuhag drivers license ddto kysa palit baril","another shooting after grimmie's incident. 50 dead. orlando, what's good? mas lisud pa guro kuhag drivers license ddto kysa palit baril",0,"another shooting after grimmie's incident. 50 dead. orlando, what's good? mas lisud pa guro kuhag drivers license ddto kysa palit baril",another shooting after grimmie's incident dead orlando what's good mas lisud pa guro kuhag drivers license ddto kysa palit baril,another shooting after grimmie's incident dead orlando what's good mas lisud pa guro kuhag drivers license ddto kysa palit baril,137,anothershootingaftergrimmie'sincidentdeadorlandowhat'sgoodmaslisudpagurokuhagdriverslicenseddtokysapalitbaril,109,another shooting after grimmie 's incident dead orlando what 's good ma lisud pa guro kuhag driver license ddto kysa palit baril
3,3,0,0,0,3,3,baptize my great nephew max tomorrow.,baptize my great nephew max tomorrow.,baptize my great nephew max tomorrow.,0,baptize my great nephew max tomorrow.,baptize my great nephew max tomorrow,baptize my great nephew max tomorrow,39,baptizemygreatnephewmaxtomorrow,31,baptize my great nephew max tomorrow
4,6,1,5,0,0,1,"RT @ovgenocide: First off, fuck your bitch. And the click you claim.","rt @ovgenocide: first off, fuck your bitch. and the click you claim.","rt @ovgenocide: first off, fuck your bitch. and the click you claim.",1,"rt : first off, fuck your bitch. and the click you claim.",rt first off fuck your bitch and the click you claim,rt first off fuck your bitch and the click you claim,57,rtfirstofffuckyourbitchandtheclickyouclaim,42,rt first off fuck your bitch and the click you claim
5,6,1,5,0,0,1,RT @____Quinn: When you thinking about killing a bitch http://t.co/bT7hf28U0d,rt @____quinn: when you thinking about killing a bitch http://t.co/bt7hf28u0d,rt @____quinn: when you thinking about killing a bitch,1,rt : when you thinking about killing a bitch,rt when you thinking about killing a bitch,rt when you thinking about killing a bitch,45,rtwhenyouthinkingaboutkillingabitch,35,rt when you think about kill a bitch
6,3,0,0,0,3,3,#psychotherapy blog: redefining what makes you,#psychotherapy blog: redefining what makes you,#psychotherapy blog: redefining what makes you,0,#psychotherapy blog: redefining what makes you,#psychotherapy blog redefining what makes you,#psychotherapy blog redefining what makes you,49,#psychotherapyblogredefiningwhatmakesyou,40,# psychotherapy blog redefine what make you
7,3,0,0,3,0,2,"'Scandal' sex scene airs on ABC right after 'It's the Great Pumpkin, Charlie Brown,' making parents angry http://t.co/vltSpK8U4S","'scandal' sex scene airs on abc right after 'it's the great pumpkin, charlie brown,' making parents angry http://t.co/vltspk8u4s","'scandal' sex scene airs on abc right after 'it's the great pumpkin, charlie brown,' making parents angry",0,"'scandal' s

##  Bridging all spaces between ashtags and apostrophes

In [1177]:
# removing spaces in between ashtags and words
df['lemmatized_1'] = df['lemmatized'].str.replace('# ', '#')
df

,count,hate_speech,offensive_language,neither,positive,class,tweet,tweet_low,no_url,handle_count,no_handle,no_handle_no_special,no_handle_no_special_no_sin_ash,tweet_length,nospaces,character_count,lemmatized,lemmatized_1
0,3,1,2,0,0,1,draymond green a bitch ass nigga bruh,draymond green a bitch ass nigga bruh,draymond green a bitch ass nigga bruh,0,draymond green a bitch ass nigga bruh,draymond green a bitch ass nigga bruh,draymond green a bitch ass nigga bruh,37,draymondgreenabitchassniggabruh,31,draymond green a bitch ass nigga bruh,draymond green a bitch ass nigga bruh
1,3,0,0,0,3,3,i decided to let you go after all. #reality,i decided to let you go after all. #reality,i decided to let you go after all. #reality,0,i decided to let you go after all. #reality,i decided to let you go after all #reality,i decided to let you go after all #reality,45,idecidedtoletyougoafterall#reality,34,i decide to let you go after all # reality,i decide to let you go after all #reality
2,3,0,0,0,3,3,"another shooting after grimmie's incident. 50 dead. orlando, what's good? mas lisud pa guro kuhag drivers license ddto kysa palit baril","another shooting after grimmie's incident. 50 dead. orlando, what's good? mas lisud pa guro kuhag drivers license ddto kysa palit baril","another shooting after grimmie's incident. 50 dead. orlando, what's good? mas lisud pa guro kuhag drivers license ddto kysa palit baril",0,"another shooting after grimmie's incident. 50 dead. orlando, what's good? mas lisud pa guro kuhag drivers license ddto kysa palit baril",another shooting after grimmie's incident dead orlando what's good mas lisud pa guro kuhag drivers license ddto kysa palit baril,another shooting after grimmie's incident dead orlando what's good mas lisud pa guro kuhag drivers license ddto kysa palit baril,137,anothershootingaftergrimmie'sincidentdeadorlandowhat'sgoodmaslisudpagurokuhagdriverslicenseddtokysapalitbaril,109,another shooting after grimmie 's incident dead orlando what 's good ma lisud pa guro kuhag driver license ddto kysa palit baril,another shooting after grimmie 's incident dead orlando what 's good ma lisud pa guro kuhag driver license ddto kysa palit baril
3,3,0,0,0,3,3,baptize my great nephew max tomorrow.,baptize my great nephew max tomorrow.,baptize my great nephew max tomorrow.,0,baptize my great nephew max tomorrow.,baptize my great nephew max tomorrow,baptize my great nephew max tomorrow,39,baptizemygreatnephewmaxtomorrow,31,baptize my great nephew max tomorrow,baptize my great nephew max tomorrow
4,6,1,5,0,0,1,"RT @ovgenocide: First off, fuck your bitch. And the click you claim.","rt @ovgenocide: first off, fuck your bitch. and the click you claim.","rt @ovgenocide: first off, fuck your bitch. and the click you claim.",1,"rt : first off, fuck your bitch. and the click you claim.",rt first off fuck your bitch and the click you claim,rt first off fuck your bitch and the click you claim,57,rtfirstofffuckyourbitchandtheclickyouclaim,42,rt first off fuck your bitch and the click you claim,rt first off fuck your bitch and the click you claim
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47778,3,2,1,0,0,0,"I'm going to sell my iPhone and both my Macs, I don't support faggots.","i'm going to sell my iphone and both my macs, i don't support faggots.","i'm going to sell my iphone and both my macs, i don't support faggots.",0,"i'm going to sell my iphone and both my macs, i don't support faggots.",i'm going to sell my iphone and both my macs i don't support faggots,i'm going to sell my iphone and both my macs i don't support faggots,70,i'mgoingtosellmyiphoneandbothmymacsidon'tsupportfaggots,55,i 'm go to sell my iphone and both my mac i do n't support faggot,i 'm go to sell my iphone and both my mac i do n't support faggot
47779,3,0,0,0,3,3,hello darkness my old friend. #scotland #sunset #hills #walking #goodbye,hello darkness my old friend. #scotland #sunset #hills #walking #goodbye,hello darkness my old frie

In [1178]:
df['lemmatized_final'] = df['lemmatized'].str.replace(" '" ,"'")
df.head(50)

,count,hate_speech,offensive_language,neither,positive,class,tweet,tweet_low,no_url,handle_count,no_handle,no_handle_no_special,no_handle_no_special_no_sin_ash,tweet_length,nospaces,character_count,lemmatized,lemmatized_1,lemmatized_final
0,3,1,2,0,0,1,draymond green a bitch ass nigga bruh,draymond green a bitch ass nigga bruh,draymond green a bitch ass nigga bruh,0,draymond green a bitch ass nigga bruh,draymond green a bitch ass nigga bruh,draymond green a bitch ass nigga bruh,37,draymondgreenabitchassniggabruh,31,draymond green a bitch ass nigga bruh,draymond green a bitch ass nigga bruh,draymond green a bitch ass nigga bruh
1,3,0,0,0,3,3,i decided to let you go after all. #reality,i decided to let you go after all. #reality,i decided to let you go after all. #reality,0,i decided to let you go after all. #reality,i decided to let you go after all #reality,i decided to let you go after all #reality,45,idecidedtoletyougoafterall#reality,34,i decide to let you go after all # reality,i decide to let you go after all #reality,i decide to let you go after all # reality
2,3,0,0,0,3,3,"another shooting after grimmie's incident. 50 dead. orlando, what's good? mas lisud pa guro kuhag drivers license ddto kysa palit baril","another shooting after grimmie's incident. 50 dead. orlando, what's good? mas lisud pa guro kuhag drivers license ddto kysa palit baril","another shooting after grimmie's incident. 50 dead. orlando, what's good? mas lisud pa guro kuhag drivers license ddto kysa palit baril",0,"another shooting after grimmie's incident. 50 dead. orlando, what's good? mas lisud pa guro kuhag drivers license ddto kysa palit baril",another shooting after grimmie's incident dead orlando what's good mas lisud pa guro kuhag drivers license ddto kysa palit baril,another shooting after grimmie's incident dead orlando what's good mas lisud pa guro kuhag drivers license ddto kysa palit baril,137,anothershootingaftergrimmie'sincidentdeadorlandowhat'sgoodmaslisudpagurokuhagdriverslicenseddtokysapalitbaril,109,another shooting after grimmie 's incident dead orlando what 's good ma lisud pa guro kuhag driver license ddto kysa palit baril,another shooting after grimmie 's incident dead orlando what 's good ma lisud pa guro kuhag driver license ddto kysa palit baril,another shooting after grimmie's incident dead orlando what's good ma lisud pa guro kuhag driver license ddto kysa palit baril
3,3,0,0,0,3,3,baptize my great nephew max tomorrow.,baptize my great nephew max tomorrow.,baptize my great nephew max tomorrow.,0,baptize my great nephew max tomorrow.,baptize my great nephew max tomorrow,baptize my great nephew max tomorrow,39,baptizemygreatnephewmaxtomorrow,31,baptize my great nephew max tomorrow,baptize my great nephew max tomorrow,baptize my great nephew max tomorrow
4,6,1,5,0,0,1,"RT @ovgenocide: First off, fuck your bitch. And the click you claim.","rt @ovgenocide: first off, fuck your bitch. and the click you claim.","rt @ovgenocide: first off, fuck your bitch. and the click you claim.",1,"rt : first off, fuck your bitch. and the click you claim.",rt first off fuck your bitch and the click you claim,rt first off fuck your bitch and the click you claim,57,rtfirstofffuckyourbitchandtheclickyouclaim,42,rt first off fuck your bitch and the click you claim,rt first off fuck your bitch and the click you claim,rt first off fuck your bitch and the click you claim
5,6,1,5,0,0,1,RT @____Quinn: When you thinking about killing a bitch http://t.co/bT7hf28U0d,rt @____quinn: when you thinking about killing a bitch http://t.co/bt7hf28u0d,rt @____quinn: when you thinking about killing a bitch,1,rt : when you thinking about killing a bitch,rt when you thinking about killing a bitch,rt when you thinking about killing a bitch,45,rtwhenyouthinkingaboutkillingabitch,35,rt when you think about kill a bitch,rt when you think about kill a bitch,rt when you think about kill a bitch
6,3,0,0,0,3,3,#psychotherapy blog: redefining what makes you,#psychotherapy blog: re

##  Removing StopWords and also words shorter than two characters

In [1179]:
# importing english stopwords
stop = stopwords.words('english')

df['tweet_without_stopwords'] = df['lemmatized'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))


In [1180]:
# checking whether stopwords are still there
df.head(10)

,count,hate_speech,offensive_language,neither,positive,class,tweet,tweet_low,no_url,handle_count,no_handle,no_handle_no_special,no_handle_no_special_no_sin_ash,tweet_length,nospaces,character_count,lemmatized,lemmatized_1,lemmatized_final,tweet_without_stopwords
0,3,1,2,0,0,1,draymond green a bitch ass nigga bruh,draymond green a bitch ass nigga bruh,draymond green a bitch ass nigga bruh,0,draymond green a bitch ass nigga bruh,draymond green a bitch ass nigga bruh,draymond green a bitch ass nigga bruh,37,draymondgreenabitchassniggabruh,31,draymond green a bitch ass nigga bruh,draymond green a bitch ass nigga bruh,draymond green a bitch ass nigga bruh,draymond green bitch ass nigga bruh
1,3,0,0,0,3,3,i decided to let you go after all. #reality,i decided to let you go after all. #reality,i decided to let you go after all. #reality,0,i decided to let you go after all. #reality,i decided to let you go after all #reality,i decided to let you go after all #reality,45,idecidedtoletyougoafterall#reality,34,i decide to let you go after all # reality,i decide to let you go after all #reality,i decide to let you go after all # reality,decide let go # reality
2,3,0,0,0,3,3,"another shooting after grimmie's incident. 50 dead. orlando, what's good? mas lisud pa guro kuhag drivers license ddto kysa palit baril","another shooting after grimmie's incident. 50 dead. orlando, what's good? mas lisud pa guro kuhag drivers license ddto kysa palit baril","another shooting after grimmie's incident. 50 dead. orlando, what's good? mas lisud pa guro kuhag drivers license ddto kysa palit baril",0,"another shooting after grimmie's incident. 50 dead. orlando, what's good? mas lisud pa guro kuhag drivers license ddto kysa palit baril",another shooting after grimmie's incident dead orlando what's good mas lisud pa guro kuhag drivers license ddto kysa palit baril,another shooting after grimmie's incident dead orlando what's good mas lisud pa guro kuhag drivers license ddto kysa palit baril,137,anothershootingaftergrimmie'sincidentdeadorlandowhat'sgoodmaslisudpagurokuhagdriverslicenseddtokysapalitbaril,109,another shooting after grimmie 's incident dead orlando what 's good ma lisud pa guro kuhag driver license ddto kysa palit baril,another shooting after grimmie 's incident dead orlando what 's good ma lisud pa guro kuhag driver license ddto kysa palit baril,another shooting after grimmie's incident dead orlando what's good ma lisud pa guro kuhag driver license ddto kysa palit baril,another shooting grimmie 's incident dead orlando 's good lisud pa guro kuhag driver license ddto kysa palit baril
3,3,0,0,0,3,3,baptize my great nephew max tomorrow.,baptize my great nephew max tomorrow.,baptize my great nephew max tomorrow.,0,baptize my great nephew max tomorrow.,baptize my great nephew max tomorrow,baptize my great nephew max tomorrow,39,baptizemygreatnephewmaxtomorrow,31,baptize my great nephew max tomorrow,baptize my great nephew max tomorrow,baptize my great nephew max tomorrow,baptize great nephew max tomorrow
4,6,1,5,0,0,1,"RT @ovgenocide: First off, fuck your bitch. And the click you claim.","rt @ovgenocide: first off, fuck your bitch. and the click you claim.","rt @ovgenocide: first off, fuck your bitch. and the click you claim.",1,"rt : first off, fuck your bitch. and the click you claim.",rt first off fuck your bitch and the click you claim,rt first off fuck your bitch and the click you claim,57,rtfirstofffuckyourbitchandtheclickyouclaim,42,rt first off fuck your bitch and the click you claim,rt first off fuck your bitch and the click you claim,rt first off fuck your bitch and the click you claim,rt first fuck bitch click claim
5,6,1,5,0,0,1,RT @____Quinn: When you thinking about killing a bitch http://t.co/bT7hf28U0d,rt @____quinn: when you thinking about killing a bitch http://t.co/bt7hf28u0d,rt @____quinn: when you thinking about killing a bitch,1,rt : when you thinking about killing a bitch,rt when you thinking about killing a bitch,rt when you thi

In [1181]:
# removing words shorter than two characters as they probably do not have much meaning
df['tweet_without_stopwords_and_2'] = df['tweet_without_stopwords'].apply(lambda x: ' '.join([word for word in x.split() if len(word)>2])) 

In [1187]:
# sanity check
df.head(5)

,count,hate_speech,offensive_language,neither,positive,class,tweet,tweet_low,no_url,handle_count,no_handle,no_handle_no_special,no_handle_no_special_no_sin_ash,tweet_length,nospaces,character_count,lemmatized,lemmatized_1,lemmatized_final,tweet_without_stopwords,tweet_without_stopwords_and_2,pos_label
0,3,1,2,0,0,1,draymond green a bitch ass nigga bruh,draymond green a bitch ass nigga bruh,draymond green a bitch ass nigga bruh,0,draymond green a bitch ass nigga bruh,draymond green a bitch ass nigga bruh,draymond green a bitch ass nigga bruh,37,draymondgreenabitchassniggabruh,31,draymond green a bitch ass nigga bruh,draymond green a bitch ass nigga bruh,draymond green a bitch ass nigga bruh,draymond green bitch ass nigga bruh,draymond green bitch ass nigga bruh,0
1,3,0,0,0,3,3,i decided to let you go after all. #reality,i decided to let you go after all. #reality,i decided to let you go after all. #reality,0,i decided to let you go after all. #reality,i decided to let you go after all #reality,i decided to let you go after all #reality,45,idecidedtoletyougoafterall#reality,34,i decide to let you go after all # reality,i decide to let you go after all #reality,i decide to let you go after all # reality,decide let go # reality,decide let reality,1
2,3,0,0,0,3,3,"another shooting after grimmie's incident. 50 dead. orlando, what's good? mas lisud pa guro kuhag drivers license ddto kysa palit baril","another shooting after grimmie's incident. 50 dead. orlando, what's good? mas lisud pa guro kuhag drivers license ddto kysa palit baril","another shooting after grimmie's incident. 50 dead. orlando, what's good? mas lisud pa guro kuhag drivers license ddto kysa palit baril",0,"another shooting after grimmie's incident. 50 dead. orlando, what's good? mas lisud pa guro kuhag drivers license ddto kysa palit baril",another shooting after grimmie's incident dead orlando what's good mas lisud pa guro kuhag drivers license ddto kysa palit baril,another shooting after grimmie's incident dead orlando what's good mas lisud pa guro kuhag drivers license ddto kysa palit baril,137,anothershootingaftergrimmie'sincidentdeadorlandowhat'sgoodmaslisudpagurokuhagdriverslicenseddtokysapalitbaril,109,another shooting after grimmie 's incident dead orlando what 's good ma lisud pa guro kuhag driver license ddto kysa palit baril,another shooting after grimmie 's incident dead orlando what 's good ma lisud pa guro kuhag driver license ddto kysa palit baril,another shooting after grimmie's incident dead orlando what's good ma lisud pa guro kuhag driver license ddto kysa palit baril,another shooting grimmie 's incident dead orlando 's good lisud pa guro kuhag driver license ddto kysa palit baril,another shooting grimmie incident dead orlando good lisud guro kuhag driver license ddto kysa palit baril,1
3,3,0,0,0,3,3,baptize my great nephew max tomorrow.,baptize my great nephew max tomorrow.,baptize my great nephew max tomorrow.,0,baptize my great nephew max tomorrow.,baptize my great nephew max tomorrow,baptize my great nephew max tomorrow,39,baptizemygreatnephewmaxtomorrow,31,baptize my great nephew max tomorrow,baptize my great nephew max tomorrow,baptize my great nephew max tomorrow,baptize great nephew max tomorrow,baptize great nephew max tomorrow,1
4,6,1,5,0,0,1,"RT @ovgenocide: First off, fuck your bitch. And the click you claim.","rt @ovgenocide: first off, fuck your bitch. and the click you claim.","rt @ovgenocide: first off, fuck your bitch. and the click you claim.",1,"rt : first off, fuck your bitch. and the click you claim.",rt first off fuck your bitch and the click you claim,rt first off fuck your bitch and the click you claim,57,rtfirstofffuckyourbitchandtheclickyouclaim,42,rt first off fuck your bitch and the click you claim,rt first off fuck your bitch and the click you claim,rt first off fuck your bitch and the click you claim,rt first fuck bitch click claim,first fuck bitch click claim,0


#  Labels for Binary Classification


In [1183]:
df['positive'].value_counts()

0    24783
3    23000
Name: positive, dtype: int64

In [1184]:
df['pos_label'] = df['class'].apply(lambda x: 1 if x == 3 else 0)

In [1185]:
df['pos_label'].value_counts()

0    24783
1    23000
Name: pos_label, dtype: int64

# Missing Values, Dropping Columns and Save for EDA

In [1188]:
df

,count,hate_speech,offensive_language,neither,positive,class,tweet,tweet_low,no_url,handle_count,no_handle,no_handle_no_special,no_handle_no_special_no_sin_ash,tweet_length,nospaces,character_count,lemmatized,lemmatized_1,lemmatized_final,tweet_without_stopwords,tweet_without_stopwords_and_2,pos_label
0,3,1,2,0,0,1,draymond green a bitch ass nigga bruh,draymond green a bitch ass nigga bruh,draymond green a bitch ass nigga bruh,0,draymond green a bitch ass nigga bruh,draymond green a bitch ass nigga bruh,draymond green a bitch ass nigga bruh,37,draymondgreenabitchassniggabruh,31,draymond green a bitch ass nigga bruh,draymond green a bitch ass nigga bruh,draymond green a bitch ass nigga bruh,draymond green bitch ass nigga bruh,draymond green bitch ass nigga bruh,0
1,3,0,0,0,3,3,i decided to let you go after all. #reality,i decided to let you go after all. #reality,i decided to let you go after all. #reality,0,i decided to let you go after all. #reality,i decided to let you go after all #reality,i decided to let you go after all #reality,45,idecidedtoletyougoafterall#reality,34,i decide to let you go after all # reality,i decide to let you go after all #reality,i decide to let you go after all # reality,decide let go # reality,decide let reality,1
2,3,0,0,0,3,3,"another shooting after grimmie's incident. 50 dead. orlando, what's good? mas lisud pa guro kuhag drivers license ddto kysa palit baril","another shooting after grimmie's incident. 50 dead. orlando, what's good? mas lisud pa guro kuhag drivers license ddto kysa palit baril","another shooting after grimmie's incident. 50 dead. orlando, what's good? mas lisud pa guro kuhag drivers license ddto kysa palit baril",0,"another shooting after grimmie's incident. 50 dead. orlando, what's good? mas lisud pa guro kuhag drivers license ddto kysa palit baril",another shooting after grimmie's incident dead orlando what's good mas lisud pa guro kuhag drivers license ddto kysa palit baril,another shooting after grimmie's incident dead orlando what's good mas lisud pa guro kuhag drivers license ddto kysa palit baril,137,anothershootingaftergrimmie'sincidentdeadorlandowhat'sgoodmaslisudpagurokuhagdriverslicenseddtokysapalitbaril,109,another shooting after grimmie 's incident dead orlando what 's good ma lisud pa guro kuhag driver license ddto kysa palit baril,another shooting after grimmie 's incident dead orlando what 's good ma lisud pa guro kuhag driver license ddto kysa palit baril,another shooting after grimmie's incident dead orlando what's good ma lisud pa guro kuhag driver license ddto kysa palit baril,another shooting grimmie 's incident dead orlando 's good lisud pa guro kuhag driver license ddto kysa palit baril,another shooting grimmie incident dead orlando good lisud guro kuhag driver license ddto kysa palit baril,1
3,3,0,0,0,3,3,baptize my great nephew max tomorrow.,baptize my great nephew max tomorrow.,baptize my great nephew max tomorrow.,0,baptize my great nephew max tomorrow.,baptize my great nephew max tomorrow,baptize my great nephew max tomorrow,39,baptizemygreatnephewmaxtomorrow,31,baptize my great nephew max tomorrow,baptize my great nephew max tomorrow,baptize my great nephew max tomorrow,baptize great nephew max tomorrow,baptize great nephew max tomorrow,1
4,6,1,5,0,0,1,"RT @ovgenocide: First off, fuck your bitch. And the click you claim.","rt @ovgenocide: first off, fuck your bitch. and the click you claim.","rt @ovgenocide: first off, fuck your bitch. and the click you claim.",1,"rt : first off, fuck your bitch. and the click you claim.",rt first off fuck your bitch and the click you claim,rt first off fuck your bitch and the click you claim,57,rtfirstofffuckyourbitchandtheclickyouclaim,42,rt first off fuck your bitch and the click you claim,rt first off fuck your bitch and the click you claim,rt first off fuck your bitch and the click you claim,rt first fuck bitch click claim,first fuck bitch click claim,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [1189]:
df.drop(['tweet','count','hate_speech','offensive_language','neither',
         'positive','class', 'tweet','no_url', 'no_handle','no_handle_no_special',
         'nospaces',], axis = 1, inplace=True)

In [1190]:
df

,tweet_low,handle_count,no_handle_no_special_no_sin_ash,tweet_length,character_count,lemmatized,lemmatized_1,lemmatized_final,tweet_without_stopwords,tweet_without_stopwords_and_2,pos_label
0,draymond green a bitch ass nigga bruh,0,draymond green a bitch ass nigga bruh,37,31,draymond green a bitch ass nigga bruh,draymond green a bitch ass nigga bruh,draymond green a bitch ass nigga bruh,draymond green bitch ass nigga bruh,draymond green bitch ass nigga bruh,0
1,i decided to let you go after all. #reality,0,i decided to let you go after all #reality,45,34,i decide to let you go after all # reality,i decide to let you go after all #reality,i decide to let you go after all # reality,decide let go # reality,decide let reality,1
2,"another shooting after grimmie's incident. 50 dead. orlando, what's good? mas lisud pa guro kuhag drivers license ddto kysa palit baril",0,another shooting after grimmie's incident dead orlando what's good mas lisud pa guro kuhag drivers license ddto kysa palit baril,137,109,another shooting after grimmie 's incident dead orlando what 's good ma lisud pa guro kuhag driver license ddto kysa palit baril,another shooting after grimmie 's incident dead orlando what 's good ma lisud pa guro kuhag driver license ddto kysa palit baril,another shooting after grimmie's incident dead orlando what's good ma lisud pa guro kuhag driver license ddto kysa palit baril,another shooting grimmie 's incident dead orlando 's good lisud pa guro kuhag driver license ddto kysa palit baril,another shooting grimmie incident dead orlando good lisud guro kuhag driver license ddto kysa palit baril,1
3,baptize my great nephew max tomorrow.,0,baptize my great nephew max tomorrow,39,31,baptize my great nephew max tomorrow,baptize my great nephew max tomorrow,baptize my great nephew max tomorrow,baptize great nephew max tomorrow,baptize great nephew max tomorrow,1
4,"rt @ovgenocide: first off, fuck your bitch. and the click you claim.",1,rt first off fuck your bitch and the click you claim,57,42,rt first off fuck your bitch and the click you claim,rt first off fuck your bitch and the click you claim,rt first off fuck your bitch and the click you claim,rt first fuck bitch click claim,first fuck bitch click claim,0
...,...,...,...,...,...,...,...,...,...,...,...
47778,"i'm going to sell my iphone and both my macs, i don't support faggots.",0,i'm going to sell my iphone and both my macs i don't support faggots,70,55,i 'm go to sell my iphone and both my mac i do n't support faggot,i 'm go to sell my iphone and both my mac i do n't support faggot,i'm go to sell my iphone and both my mac i do n't support faggot,'m go sell iphone mac n't support faggot,sell iphone mac n't support faggot,0
47779,hello darkness my old friend. #scotland #sunset #hills #walking #goodbye,0,hello darkness my old friend #scotland #sunset #hills #walking #goodbye,77,62,hello darkness my old friend # scotland # sunset # hill # walk # goodbye,hello darkness my old friend #scotland #sunset #hill #walk #goodbye,hello darkness my old friend # scotland # sunset # hill # walk # goodbye,hello darkness old friend # scotland # sunset # hill # walk # goodbye,hello darkness old friend scotland sunset hill walk goodbye,1
47780,i can't believe that my antidepressants are working. i didn't know if they were going to work or not.,0,i can't believe that my antidepressants are working i didn't know if they were going to work or not,103,81,i ca n't believe that my antidepressant be work i do n't know if they be go to work or not,i ca n't believe that my antidepressant be work i do n't know if they be go to work or not,i ca n't believe that my antidepressant be work i do n't know if they be go to work or not,ca n't believe antidepressant work n't know go work,n't believe antidepressant work n't know work,1
47781,"#porn,#android,#iphone,#ipad,#sex,#xxx, | #anal | hardcore british queer anal pounding http://t.co/lrueixmy21",0,#porn #android #iphone #ipad #sex #xxx #anal hardcore british q

In [1192]:
df.isna().sum()

tweet_low                          0
handle_count                       0
no_handle_no_special_no_sin_ash    0
tweet_length                       0
character_count                    0
lemmatized                         0
lemmatized_1                       0
lemmatized_final                   0
tweet_without_stopwords            0
tweet_without_stopwords_and_2      0
pos_label                          0
dtype: int64

In [1193]:
len(df)

47783

In [1194]:
# saving file
df.to_csv(r'../data/processed/cleaned_data_2.csv', index = False)